In [1]:
import pandas as pd
import nltk
from sklearn.utils import shuffle

# Loading our datafile as a DataFrame with pandas

df = pd.read_csv('/Users/laniepreston/TMCI_Edits/group_assignment/FullReviews.csv' , usecols=[1,6,9], header=0)
shuf_red_df = shuffle(df[:10000])
print(shuf_red_df)

       ProductId  Score                                               Text
2945  B0002TJAZK      3  I ordered this because it is Science Diet and ...
1929  B0016PKA1C      1  I've been told that the refrigerator of the fu...
8339  B003VXFK44      2  The coffee is good. However, at least half of ...
2567  B000ED9LDU      3  It was my mistake - I thought there were 6 bag...
309   B003YDP5PA      5  This tri-color couscous is really good...full ...
8874  B001EQ5EQE      4  I've been using this coffee for several years ...
1388  B00068PCTU      5  If you need to cut down on sugar this is the w...
1161  B003JNWQPC      1  This vendor is unscrupulous. She ripped off he...
3751  B00016UX0K      5  Here's a tiny consolation: easy Thai barbecued...
1894  B001RVFDOO      5  I love these chips. They are so delicious, it ...
1881  B001RVFDOO      5  These were awesome!! They have a great crunchy...
173   B0081XN2HQ      5  I couldn't wait to try the 8&#8243; Cinnamon C...
8994  B006N3IG4K      5  

In [4]:
# PoS-tagging our dataset
train_df = shuf_red_df[:8000]
test_df = shuf_red_df[:2000]


train_tagged = []
test_tagged = []


for index, item in train_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    train_tagged.append([rev, score])

for index, item in test_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    test_tagged.append([rev, score])
    

In [5]:
print(test_tagged[:10])

[[[('I', 'PRON'), ('ordered', 'VERB'), ('this', 'DET'), ('because', 'ADP'), ('it', 'PRON'), ('is', 'VERB'), ('Science', 'NOUN'), ('Diet', 'ADJ'), ('and', 'CONJ'), ('it', 'PRON'), ('says', 'VERB'), ('it', 'PRON'), ('is', 'VERB'), ('minced.', 'NOUN'), ('Well,', 'NOUN'), ('it', 'PRON'), ('is', 'VERB'), ('NOT', 'NOUN'), ('minced', 'VERB'), ('and', 'CONJ'), ('my', 'PRON'), ('cats', 'NOUN'), ('do', 'VERB'), ('not', 'ADV'), ('like', 'ADP'), ('it.', 'NOUN'), ('It', 'PRON'), ('is', 'VERB'), ('more', 'ADV'), ('of', 'ADP'), ('a', 'DET'), ('pate', 'NOUN'), ('than', 'ADP'), ('minced', 'VERB'), ('and', 'CONJ'), ('gets', 'VERB'), ('mashed', 'VERB'), ('down', 'ADV'), ('in', 'ADP'), ('their', 'PRON'), ('bowl', 'NOUN'), ('so', 'ADV'), ('they', 'PRON'), ("can't", 'NOUN'), ('eat', 'NOUN'), ('it.', 'NOUN'), ('I', 'PRON'), ('am', 'VERB'), ('very', 'ADV'), ('unhappy', 'ADJ'), ('with', 'ADP'), ('this', 'DET'), ('product', 'NOUN'), ('and', 'CONJ'), ('my', 'PRON'), ('cats', 'NOUN'), ('are', 'VERB'), ('too.', 'N

In [2]:
cooccs_text = []
for index, item in shuf_red_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
        cooccs_text.append([product_id, rev, score])

In [3]:
print(cooccs_text[:10])

[['B0002TJAZK', [('I', 'PRON'), ('ordered', 'VERB'), ('this', 'DET'), ('because', 'ADP'), ('it', 'PRON'), ('is', 'VERB'), ('Science', 'NOUN'), ('Diet', 'ADJ'), ('and', 'CONJ'), ('it', 'PRON'), ('says', 'VERB'), ('it', 'PRON'), ('is', 'VERB'), ('minced.', 'NOUN'), ('Well,', 'NOUN'), ('it', 'PRON'), ('is', 'VERB'), ('NOT', 'NOUN'), ('minced', 'VERB'), ('and', 'CONJ'), ('my', 'PRON'), ('cats', 'NOUN'), ('do', 'VERB'), ('not', 'ADV'), ('like', 'ADP'), ('it.', 'NOUN'), ('It', 'PRON'), ('is', 'VERB'), ('more', 'ADV'), ('of', 'ADP'), ('a', 'DET'), ('pate', 'NOUN'), ('than', 'ADP'), ('minced', 'VERB'), ('and', 'CONJ'), ('gets', 'VERB'), ('mashed', 'VERB'), ('down', 'ADV'), ('in', 'ADP'), ('their', 'PRON'), ('bowl', 'NOUN'), ('so', 'ADV'), ('they', 'PRON'), ("can't", 'NOUN'), ('eat', 'NOUN'), ('it.', 'NOUN'), ('I', 'PRON'), ('am', 'VERB'), ('very', 'ADV'), ('unhappy', 'ADJ'), ('with', 'ADP'), ('this', 'DET'), ('product', 'NOUN'), ('and', 'CONJ'), ('my', 'PRON'), ('cats', 'NOUN'), ('are', 'VERB'

In [22]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
# Lemmatizing our dataset + filtering for stopwords (except negation)
training_dicts = dict((k,[]) for k in [1,2,3,4,5])
test_dicts = dict((k,[]) for k in [1,2,3,4,5])

stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

lemmatized = []

for review in train_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
            else: 
                continue
        
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    training_dicts[score].append(lem_rev)

In [11]:
test_dicts = dict((k,[]) for k in [1,2,3,4,5])
for review in test_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    test_dicts[score].append(lem_rev)

In [8]:
cooccs_dict = {}
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

for review in cooccs_text:
    product_id = review[0]
    rev = review[1]
    score = review[2]
    lem_rev = []
    
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
        
    cooccs_dict[product_id] = (score, lem_rev)

In [12]:
print(cooccs_dict)

{'B0002TJAZK': (3, ['food-NOUN', 'work-VERB', 'well-ADV', 'kidney-NOUN', 'find-VERB', 'info-NOUN', 'many-ADJ', 'can-NOUN', 'include-VERB', 'guess-VERB', 'can-NOUN', 'base-VERB', 'shipping-NOUN', 'weight-NOUN', 'buy-VERB', 'without-ADP', 'well-ADV']), 'B0016PKA1C': (3, ['get-VERB', 'one-NUM', 'honest-NOUN', 'work-NOUN', 'well-ADV', 'extend-NOUN', 'life-NOUN', 'fresh-ADJ', 'herb-NOUN', 'provide-VERB', 'attractive-ADJ', 'main-ADJ', 'problem-NOUN', 'container-NOUN', 'difficult-ADJ', 'get-VERB', 'easy-ADJ', 'time-NOUN', 'whoever-NOUN', 'design-VERB', 'thing-NOUN', 'not-NEGATION', 'not-NEGATION', 'make-VERB', 'hard-ADJ', 'get-VERB', 'thing-NOUN', 'easy-ADJ', 'open-ADJ', 'nothing-NOUN', 'inside-ADP', 'use-NOUN', 'much-ADJ', 'force-NOUN', 'need-VERB', 'open-ADJ', 'need-NOUN', 'open-ADJ', 'squeeze-NOUN', 'side-NOUN', 'pull-NOUN', 'plastic-NOUN', 'cover-NOUN', 'break-NOUN', 'pretty-ADV', 'sturdy-NOUN', 'real-ADJ', 'issue-NOUN', 'fill-VERB', 'water-NOUN', 'difficult-ADJ', 'use-NOUN', 'much-ADJ', 

In [13]:
print(test_dicts)

{1: [['told-NOUN', 'refrigerator-NOUN', 'future-NOUN', 'bin-NOUN', 'inside-ADP', 'grow-NOUN', 'vegetable-NOUN', 'thought-NOUN', 'prepara-NOUN', 'herb-NOUN', 'savor-NOUN', 'pod-NOUN', 'look-VERB', 'like-ADP', 'perfect-NOUN', 'detachable-ADJ', 'bottom-NOUN', 'come-VERB', 'loose-ADJ', 'continually-ADV', 'leak-NOUN', 'water-NOUN', 'plastic-NOUN', 'plug-NOUN', 'back-ADV', 'not-NEGATION', 'not-NEGATION', 'good-ADJ', 'poorly-ADV', 'refrain-NOUN', 'purchasing-NOUN', 'plastic-NOUN', 'bag-NOUN', 'much-ADJ'], ['vendor-NOUN', 'ripped-NOUN', 'culture-NOUN', 'collector-NOUN', 'shamelessly-ADV', 'claim-NOUN', 'no-NEGATION', 'no-NEGATION', 'conscience-NOUN', 'would-VERB', 'highly-ADV', 'recommend-NOUN', 'purchasing-NOUN'], ['powder-NOUN', 'spoilt-NOUN', 'lose-VERB', 'raise-VERB', 'still-ADV', 'well-ADV', 'within-ADP', 'expiry-NOUN', 'date-NOUN', 'container-NOUN', 'kept-NOUN', 'close-VERB', 'dry-NOUN', 'avoid-NOUN', 'plan-NOUN', 'ask-NOUN'], ['first-ADV', 'love-NOUN', 'whole-ADJ', 'wheat-NOUN', 'flax-N

In [15]:
import pickle
# Save lemmatized text to a pickle file
#pickle.dump(training_dicts, open("training_dicts.txt", "wb"))
pickle.dump(test_dicts, open("test_dicts.txt", "wb"))
pickle.dump(cooccs_dict, open("cooccs_dict.txt", "wb"))

In [18]:
print(cooccs_text[:5])

[['B00474H936', [('I', 'PRON'), ('have', 'VERB'), ('tried', 'VERB'), ('almost', 'ADV'), ('every', 'DET'), ('kind', 'NOUN'), ('of', 'ADP'), ('"fake', 'NOUN'), ('meat"', 'NOUN'), ('product', 'NOUN'), ('out', 'ADP'), ('there.', 'NOUN'), ('While', 'ADP'), ('I', 'PRON'), ('really', 'ADV'), ('like', 'ADP'), ('the', 'DET'), ('Morning', 'VERB'), ('Stars', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('Boca', 'NOUN'), ('"ground', 'NOUN'), ('meat"', 'NOUN'), ('you', 'PRON'), ("can't", 'NOUN'), ('really', 'ADV'), ('make', 'VERB'), ('them', 'PRON'), ('into', 'ADP'), ('anything.<br', 'NOUN'), ('/><br', 'NOUN'), ('/>This', 'NOUN'), ('product', 'NOUN'), ('was', 'VERB'), ('great.', 'NOUN'), ('Soak', 'NOUN'), ('it', 'PRON'), ('for', 'ADP'), ('15', 'NUM'), ('minutes', 'NOUN'), ('in', 'ADP'), ('water,', 'NOUN'), ('and', 'CONJ'), ('viola', 'NOUN'), ('you', 'PRON'), ('have', 'VERB'), ('a', 'DET'), ('really', 'ADV'), ('good', 'ADJ'), ('ground', 'NOUN'), ('meat', 'NOUN'), ('you', 'PRON'), ('can', 'VERB'), ('mak